In [1]:
import pandas as pd

df = pd.read_csv('./address.csv')
df.head()

,total_region,city1,city2,country,village
0,서울특별시,NaN,서울 서울시 서울특별시,NaN,NaN
1,서울,NaN,서울 서울시 서울특별시,NaN,NaN
2,서울시,NaN,서울 서울시 서울특별시,NaN,NaN
3,서울특별시 종로구,NaN,서울 서울시 서울특별시,종로구,NaN
4,서울특별시 종로구 청운효자동,NaN,서울 서울시 서울특별시,종로구,청운효자동


In [48]:
len(df)

4473

In [2]:
import os
import sys
import urllib.request

client_id = "Ac3AoE6vHArCjuG2aTjg"
client_secret = "3m9UJuTDLa"

def trans(text):
    encText = urllib.parse.quote(text)
    data = "source=ko&target=en&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        result = response_body.decode('utf-8')
        return result['message']['result']['translatedText']
    else:
        print(text)
        print("Error Code:" + rescode)
        return None

In [5]:
origin, translated = [], []
for i, text in enumerate(df['total_region'].tolist()):
    result = trans(text)
    if result:
        origin.append(text)
        translated.append(result)
    else:
        print(i)
        break

HTTPError: HTTP Error 429: Too Many Requests

In [25]:
translated = [json.loads(t)['message']['result']['translatedText'] for t in translated]

In [28]:
len(origin)

730

In [29]:
pd.DataFrame({'orogin': origin, 'translated': translated}).to_csv('./region_kor_eng1.csv', index=False, encoding='utf-8')

In [35]:
from selenium import webdriver
import random
import time

def trans_crawl(text):
    try:
        driver = webdriver.Chrome("../chromedriver")
        driver.get("https://papago.naver.com")
        time.sleep(random.uniform(1,2))
        input_box = driver.find_element_by_css_selector("textarea#txtSource")
        input_box.send_keys(text)
        button = driver.find_element_by_css_selector("button#btnTranslate")
        button.click()
        time.sleep(random.uniform(1,2))
        result = driver.find_element_by_css_selector("div#txtTarget").text
        driver.close()
        return result
    except:
        return None

In [36]:
print(trans_crawl(df['total_region'].tolist()[730]))

Yongdang-dong, Nam-gu, Busan


In [59]:
origin, translated = [], []
for i, text in enumerate(df['total_region'].tolist()[730+1032+657+20+1806+164:]):
    result = trans_crawl(text)
    if result:
        origin.append(text)
        translated.append(result)
    else:
        print(i)
        break

In [61]:
730+1032+657+20+1806+164+64

4473

In [60]:
print(len(origin), len(translated))

64 64


In [62]:
i = 7

In [63]:
pd.DataFrame({'origin': origin, 'translated': translated}).to_csv('./region_kor_eng'+ str(i) +'.csv', index=False, encoding='utf-8')